Hope you enjoy! 

Cheers,
kyle

try: **conda install psycopg2** or **pip install psycopg2**

don't forget insert your **password!!!**

In [1]:
import psycopg2
import math
from random import randint
import random
from itertools import count
import time
from datetime import date


In [2]:
class TableDataGen(object):
    """
    Instatiate Class with Num of Rows of data you want for a table, the 'Table' name.
    Call PKs first, then gen_names, then gen_emails
    """
    def __init__(self, num_rows, table):
        self.num_rows = num_rows
        self.table = table #what table in cats you are inserting data into
#         self.foriegn_keys = foriegn_key_limits #if the table has FKs, make sure not to exceed their values
        self.data = [] #data to export
        
    def get_size(self, s):
        return len(s.encode('utf-8'))
        
    def gen_names(self):
        with open('names.txt', 'r') as f:
            random_names = []
            random_element = [x.split() for x in f.readlines()][0] #silly unpacking hack... [0]
            for i in range(1,self.num_rows):
                random_names.append(random.choice(random_element))
            return self.data.append(random_names)

    
    def gen_emails(self):
        with open('names.txt', 'r') as f:
            random_emails = []
            random_element = [x.split() for x in f.readlines()][0]
            for i in range(1, self.num_rows):
                random_emails.append(random.choice(random_element).lower()+'01@make_my_own_data.opt')
            self.data.append(random_emails)
    
    def gen_primary_keys(self):
        return self.data.append(range(1, self.num_rows))
    
    def gen_foreign_keys(self, sql):
        cur.execute(sql)
        rows = cur.fetchall()
        fk_choice =[]
        for row in rows:
            fk_choice.append(*list(row))
        new_fk = []
        for e in range(1, self.num_rows):
            new_fk.append(random.choice(fk_choice))
        self.data.append(new_fk)
        
    
    def gen_friend_pairings(self):
        base_users = range(1, self.num_rows) #random sampling from PKs
        friend_pairs = set() #set to ensure no duplicates
        for pairs in range(1, 20*len(base_users)): #we want 20 times the friend pairings as PKs
            user1, user2 = random.choice(base_users), random.choice(base_users) #create friends
            if user1 == user2: #you can't be your own friend in this universe
                continue
            friend_pairs.add((user1,user2))
        friend_pairs = zip(*list(friend_pairs)) #unpack tuple pairs into 2 lists
        return self.data.extend(friend_pairs) #dump it into data
        
    
    def gen_video_names(self):
        video_names = []
        for video in range(1, self.num_rows):
            video_names.append('vid:'+str(randint(1,10000)))
        return self.data.append(video_names)
    
#     def gen_like_id_package(self):
#         ids = range(1, self.num_rows)
#         id_pool = range(1, len(ids)*40) #40 times likes as users
#         self.data.append(id_pool)

#         like_pairs = set()
#         for pairs in id_pool: #create a like pair for each id_pool
#             user, video = random.choice(ids), random.choice(ids)
#             like_pairs.add((user,video))
#         x = zip(*list(like_pairs))
#         x = list(x[0]),list(x[1])
#         self.data.extend(x)
        
    def random_date(self, start, end, format, prop):
        stime = time.mktime(time.strptime(start, format))
        etime = time.mktime(time.strptime(end, format))
        ptime = stime + prop * (etime - stime)
        return time.strftime(format, time.localtime(ptime))
            
    def gen_timestamp(self):
        times = []
        for rows in range(1, len(self.data[0])):
            times.append(self.random_date("1/1/2015 1:30", "1/1/2016 4:50", '%m/%d/%Y %I:%M', random.random()))
        self.data.append(times)
        

    def insert_data(self, col, col_insert):
#         print self.data
        insert_query = """INSERT INTO cats.{}{} VALUES {}""".format(self.table, col, col_insert)
        for row in zip(*self.data):
            cur.execute(insert_query, row)


### insert password here:

In [3]:
# create a connection to database. make sure that the db name and password are correct
try:
    conn = psycopg2.connect("dbname='DSE201_DBs' user='postgres' host='/tmp/' password='Awesome534'")
except:
    print "unable to connect to the database"
    
    
conn.set_session(autocommit = True) #else use conn.commit() for each insert statement call
cur = conn.cursor()

Delete statements if you want to flush the DB, open up query tool and copy pasta them in and execute.

In [4]:
# Delete all data from cats:

# DELETE FROM cats.suggestion;
# DELETE FROM cats.watch;
# DELETE FROM cats.friend;
# DELETE FROM cats.likes;
# DELETE FROM cats.login;
# DELETE FROM cats.user;
# DELETE FROM cats.video;


- 5000 users
- 3000 videos
- 20000 friend combos
- 10000 likes
- 20000 logins
- 20000 watches
- 5000 suggestions

In [5]:
user = TableDataGen(5000, 'user')
user.gen_primary_keys()
user.gen_names()
user.gen_emails()
user.insert_data('(user_id,user_name,facebook_id)', '(%s,%s,%s)')

In [6]:
video = TableDataGen(3000, 'video')
video.gen_primary_keys()
video.gen_video_names()
video.insert_data('(video_id,video_name)', '(%s,%s)')

In [7]:
friend = TableDataGen(1000, 'friend') #creating 20 times!!! friend pairings for num of users
friend.gen_friend_pairings()
friend.insert_data('(user_id,friend_id)', '(%s,%s)')

In [8]:
likes = TableDataGen(10000, 'likes')
# likes.gen_like_id_package()
likes.gen_primary_keys()
likes.gen_foreign_keys("""SELECT user_id FROM cats.user""")
likes.gen_foreign_keys("""SELECT video_id FROM cats.video""")
likes.gen_timestamp()
likes.insert_data('(like_id,user_id,video_id,time)', '(%s,%s,%s,%s)')

In [9]:
login = TableDataGen(20000, 'login')
login.gen_primary_keys()
login.gen_foreign_keys("""SELECT user_id FROM cats.user""")
login.gen_timestamp()
login.insert_data('(login_id,user_id,time)', '(%s,%s,%s)')

In [10]:
watch = TableDataGen(20000, 'watch')
watch.gen_primary_keys()
watch.gen_foreign_keys("""SELECT video_id FROM cats.video""")
watch.gen_foreign_keys("""SELECT user_id FROM cats.user""")
watch.gen_timestamp()
watch.insert_data('(watch_id,video_id,user_id,time)', '(%s,%s,%s,%s)')

In [11]:
suggestion = TableDataGen(5000, 'suggestion')
suggestion.gen_primary_keys()
suggestion.gen_foreign_keys("""SELECT login_id FROM cats.login""")
suggestion.gen_foreign_keys("""SELECT video_id FROM cats.video""")
suggestion.insert_data('(suggestion_id,login_id,video_id)', '(%s,%s,%s)')

### Test out the data quality by pulling from the database:

In [12]:
# Test DB Queries:
tests = ['user','video','friend','likes', 'suggestion', 'watch','login'] #get data from base tables
for test in tests:
    try:
        cur.execute("""SELECT * from cats.{} limit 10""".format(test))
    except:
        print "Failed SELECT from cats.{}".format(test)

    rows = cur.fetchall()
    print '\nLIMIT 10 from table:  ----- {} -----'.format(test)
    for row in rows:
        print row


LIMIT 10 from table:  ----- user -----
(1, 'Ashlie', 'sammy01@make_my_own_data.opt')
(2, 'Edwardo', 'elsa01@make_my_own_data.opt')
(3, 'Shante', 'alaina01@make_my_own_data.opt')
(4, 'Marla', 'carie01@make_my_own_data.opt')
(5, 'Drucilla', 'jeffery01@make_my_own_data.opt')
(6, 'Eve', 'dan01@make_my_own_data.opt')
(7, 'Noble', 'jayson01@make_my_own_data.opt')
(8, 'Cherise', 'madlyn01@make_my_own_data.opt')
(9, 'Augustine', 'felton01@make_my_own_data.opt')
(10, 'Kaila', 'dean01@make_my_own_data.opt')

LIMIT 10 from table:  ----- video -----
(1, 'vid:6872')
(2, 'vid:6696')
(3, 'vid:5097')
(4, 'vid:2835')
(5, 'vid:8214')
(6, 'vid:4177')
(7, 'vid:6890')
(8, 'vid:6305')
(9, 'vid:4033')
(10, 'vid:6723')

LIMIT 10 from table:  ----- friend -----
(780, 725)
(485, 297)
(685, 525)
(627, 464)
(167, 205)
(794, 141)
(613, 45)
(760, 751)
(725, 611)
(691, 943)

LIMIT 10 from table:  ----- likes -----
(1, 2264, 1021, datetime.datetime(2015, 12, 20, 9, 49))
(2, 3549, 2074, datetime.datetime(2015, 5, 2, 

In [13]:
#close the connection to db
conn.close()